In [1]:
import juniper
import scanpy as sc
import os

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import scvi
import pandas as pd

In [110]:
df_hyper = pd.read_csv("result_hyper_factorvi_combosciplex.csv")

In [111]:
df_hyper.sort_values("r2_ood")

,Unnamed: 0,validation_loss,elbo_validation,reconstruction_loss_validation,kl_local_validation,kl_global_validation,disentangling_loss_validation,train_loss,rec_loss,kld_loss,elbo_train,reconstruction_loss_train,kl_local_train,kl_global_train,disentangling_loss_train,r2_train,r2_ood
5,"32_0.5_1.0_[1024, 1024]",80.496242,102.396545,73.768692,28.627855,0.0,0.0,82.658617,75.603628,30.021231,105.624855,75.603622,30.021231,0.0,0.0,0.983327,0.982233
15,"32_1.0_1.0_[1024, 1024]",80.737576,98.845085,73.782791,25.062294,0.0,0.0,82.971842,75.584645,26.620529,102.205185,75.584656,26.620529,0.0,0.0,0.985752,0.983165
16,"32_1.0_1.0_[1024, 1024, 1024]",81.049779,93.764633,75.469879,18.294758,0.0,0.0,83.096097,77.136607,19.539309,96.675919,77.136612,19.539309,0.0,0.0,0.986006,0.983398
9,"32_0.5_1.0_[4096, 4096]",79.940723,100.949516,72.937798,28.011719,0.0,0.0,80.801470,72.674686,32.507141,105.181816,72.674675,32.507141,0.0,0.0,0.986563,0.984017
7,"32_0.5_1.0_[2048, 2048]",80.289152,101.231224,72.835197,28.396025,0.0,0.0,81.447825,73.552989,30.075569,103.628563,73.552994,30.075569,0.0,0.0,0.986769,0.984692
6,"32_0.5_1.0_[1024, 1024, 1024]",80.877219,93.362007,75.462669,17.899340,0.0,0.0,82.613163,76.753623,19.370380,96.124008,76.753624,19.370380,0.0,0.0,0.987255,0.985120
17,"32_1.0_1.0_[2048, 2048]",79.777485,101.721954,72.462662,29.259296,0.0,0.0,81.438829,73.644614,31.176865,104.821480,73.644615,31.176865,0.0,0.0,0.987244,0.985311
18,"32_1.0_1.0_[2048, 2048, 2048]",79.808070,98.296959,73.311981,24.984980,0.0,0.0,80.783648,73.987052,26.140757,100.127808,73.987053,26.140757,0.0,0.0,0.987245,0.985584
19,"32_1.0_1.0_[4096, 4096]",79.714091,99.021248,72.112312,26.908936,0.0,0.0,81.080492,72.518124,30.309271,102.827400,72.518127,30.309271,0.0,0.0,0.989648,0.987043
8,"32_0.5_1.0_[2048, 2048, 2048]",79.305397,96.752426,72.933960,23.818464,0.0,0.0,80.964431,74.160855,25.433929,99.594788,74.160858,25.433929,0.0,0.0,0.989681,0.987641


In [112]:
def split_string(s):
    split_data = s.split('_')
    split_data[-1] = split_data[-1]
    return split_data

In [113]:
split_columns = df_hyper['Unnamed: 0'].apply(split_string)

In [114]:
split_df = pd.DataFrame(split_columns.tolist(), columns=["latent_dim", "kl", "disentanglement", "layers"])
result_df = df_hyper.join(split_df)

In [115]:
result_df = result_df[result_df["r2_train"]>0.0]

In [116]:
result_df.groupby("layers").mean(["r2_ood"])

,validation_loss,elbo_validation,reconstruction_loss_validation,kl_local_validation,kl_global_validation,disentangling_loss_validation,train_loss,rec_loss,kld_loss,elbo_train,reconstruction_loss_train,kl_local_train,kl_global_train,disentangling_loss_train,r2_train,r2_ood
layers,,,,,,,,,,,,,,,,
"[1024, 1024, 1024]",75.311400,107.686966,71.189616,36.497351,0.0,0.0,76.667477,72.299898,38.215780,110.515681,72.299901,38.215780,0.0,0.0,0.990382,0.988453
"[1024, 1024]",75.208455,113.750742,70.380458,43.370285,0.0,0.0,76.285579,71.266593,44.579934,115.846532,71.266599,44.579934,0.0,0.0,0.989287,0.987994
"[2048, 2048, 2048]",75.138910,112.761881,70.505041,42.256839,0.0,0.0,75.764782,70.919601,43.352794,114.272396,70.919603,43.352794,0.0,0.0,0.990745,0.989265
"[2048, 2048]",75.249985,118.121637,70.142000,47.979637,0.0,0.0,76.191744,70.785940,50.289658,121.075603,70.785946,50.289658,0.0,0.0,0.989886,0.988153
"[4096, 4096]",75.000525,114.112625,69.733032,44.379594,0.0,0.0,75.088089,69.148786,49.236962,118.385746,69.148783,49.236962,0.0,0.0,0.991150,0.989127


In [117]:
result_df.groupby("latent_dim").mean(["r2_ood"])

,validation_loss,elbo_validation,reconstruction_loss_validation,kl_local_validation,kl_global_validation,disentangling_loss_validation,train_loss,rec_loss,kld_loss,elbo_train,reconstruction_loss_train,kl_local_train,kl_global_train,disentangling_loss_train,r2_train,r2_ood
latent_dim,,,,,,,,,,,,,,,,
32,75.188952,113.053323,70.425939,42.627384,0.0,0.0,76.042897,70.965899,44.795283,115.761184,70.965901,44.795283,0.0,0.0,0.990248,0.988565


In [ ]:
result_df[result_df["latent_dim"]=="32"].sort_values("r2_ood", ascending=False).head(5)

In [ ]:
result_df[result_df["latent_dim"]=="64"].sort_values("r2_ood", ascending=False).head(5)

In [ ]:
result_df[result_df["latent_dim"]=="128"].sort_values("r2_ood", ascending=False).head(5)

In [ ]:
result_df[result_df["latent_dim"]=="256"].sort_values("r2_ood", ascending=False).head(5)

In [ ]:
result_df[result_df["latent_dim"]=="512"].sort_values("r2_ood")

In [ ]:
result_df.groupby("disentanglement").mean(["r2_ood"])

In [11]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex"

In [12]:
adata_train = sc.read(os.path.join(output_dir, "adata_train_30.h5ad"))
adata_test = sc.read(os.path.join(output_dir, "adata_test_30.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, "adata_ood_30.h5ad")) 

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [43]:
juniper.latent.model.FactorVI.setup_anndata(adata_train)
vae = juniper.latent.model.FactorVI(adata_train, count_input=False, n_latent = 32, encoder_layers= [2048, 2048], decoder_layers = [2048, 2048], disentangling_weight=0.1, kld_weight=0.5)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [44]:
vae.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=20,
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-92163b8d-2d3b-5999-bfa6-7aaea52601b1]
/ictstr01/home/

Epoch 146/2000:   7%|▋         | 146/2000 [04:23<55:48,  1.81s/it, v_num=1, train_loss=77.5, rec_loss=70.8, kld_loss=37.3]  
Monitored metric reconstruction_loss_validation did not improve in the last 20 records. Best score: 69.505. Signaling Trainer to stop.


In [45]:
adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train)

In [46]:
adata_train.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_train, give_mean=True)

In [47]:
from sklearn.metrics import pairwise_distances, r2_score
import numpy as np

In [48]:
r2_score(np.mean(adata_train.obsm["reconstruction"], axis=0), np.mean(adata_train.X.A, axis=0))

0.9909998884841147

In [49]:
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)
adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood, give_mean=True)
r2_ood = r2_score(np.mean(adata_ood.obsm["reconstruction"], axis=0), np.array(np.mean(adata_ood.X, axis=0)).squeeze())


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [50]:
r2_ood

0.9903856653078081

In [51]:
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test, give_mean=True)
adata_test.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_test, give_mean=True)
r2_score(np.mean(adata_test.obsm["reconstruction"], axis=0), np.array(np.mean(adata_test.X.A, axis=0)))

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


0.9907811483734912

In [52]:
save_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/combosciplex_factorvi_test"
vae.save(save_path, overwrite=True)

In [53]:
adata_train.write(os.path.join(output_dir, "adata_train_30_with_scvi_32.h5ad"))
adata_test.write(os.path.join(output_dir, "adata_test_30_with_scvi_32.h5ad"))
adata_ood.write(os.path.join(output_dir, "adata_ood_30_with_scvi_32.h5ad")) 

In [54]:
adata_test.X.sum(axis=1)

matrix([[104.06918 ],
        [113.903946],
        [138.54303 ],
        ...,
        [127.169205],
        [168.24808 ],
        [168.16222 ]], dtype=float32)

In [55]:
adata_test.obsm["reconstruction"].sum(axis=1)

array([ 90.1851  , 104.84177 , 118.521614, ..., 107.94208 , 150.06708 ,
       147.20398 ], dtype=float32)

In [80]:
adata_ood = sc.read(os.path.join(output_dir, "adata_ood_30.h5ad")) 

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [101]:
adata_ood2 = adata_ood.copy()

In [102]:
adata_ood2.obsm["X_scVI"] = vae.get_latent_representation(adata_ood2)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [103]:
adata_ood2.X = np.zeros(adata_ood.shape)

In [104]:
adata_ood2.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood2, give_mean=True)

In [105]:
adata_ood2.obsm["reconstruction"].sum(axis=1)

array([151.72835, 135.42264, 160.055  , ..., 158.63513, 164.88205,
       112.80001], dtype=float32)

In [100]:
np.unique(adata_ood.X[5].A, return_counts=True)

(array([0.        , 0.80856794, 1.2497247 , 1.554786  , 1.788214  ,
        1.9773334 , 2.1363118 , 2.501618  , 4.166343  , 4.4206414 ],
       dtype=float32),
 array([1862,  103,   17,    6,    6,    1,    2,    1,    1,    1]))

In [78]:
del adata_ood3.obsm["X_scVI"]

In [71]:
adata_ood2.obsm["reconstruction"]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
adata_ood2.obsm["reconstruction"].sum(axis=1)

array([16.44633 , 16.44633 , 16.44633 , ..., 16.478039, 16.478039,
       16.478039], dtype=float32)

In [60]:
adata_ood.obsm["reconstruction"].sum(axis=1)

array([151.72835, 135.42264, 160.055  , ..., 158.63513, 164.88205,
       112.80001], dtype=float32)

matrix([[169.3923 ],
        [154.38382],
        [180.55472],
        ...,
        [170.18831],
        [172.17686],
        [136.98788]], dtype=float32)

In [63]:
adata_ood2.obsm["reconstruction2"] = vae.get_reconstructed_expression(adata_ood2, give_mean=True)

In [64]:
adata_ood2.obsm["reconstruction2"].sum(axis=1)

array([16.44633 , 16.44633 , 16.44633 , ..., 16.478039, 16.478039,
       16.478039], dtype=float32)

In [80]:
juniper.latent.model.FactorVI.setup_anndata(adata_train)
vae_128 = juniper.latent.model.FactorVI(adata_train, n_latent = 128, encoder_layers= [1024, 1024, 1024], decoder_layers = [1024, 1024, 1024], disentangling_weight=0.1, kld_weight=2.0)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [81]:
vae_128.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=20,
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_geno ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/ictstr01/home/icb/dominik.klein/mambaforge/envs/ot_pe

Epoch 311/2000:  16%|██           | 311/2000 [11:55<1:04:43,  2.30s/it, v_num=1, train_loss=499, rec_loss=487, kld_loss=7.6]
Monitored metric reconstruction_loss_validation did not improve in the last 20 records. Best score: 484.747. Signaling Trainer to stop.


In [93]:
adata_train.obsm["X_scVI"] = vae_128.get_latent_representation(adata_train)

In [94]:
adata_train.obsm["reconstruction"] = vae_128.get_reconstructed_expression(adata_train, give_mean=True)

In [96]:
r2_score(np.mean(np.log1p(adata_train.obsm["reconstruction"]), axis=0), np.mean(np.log1p(adata_train.X.A), axis=0))

0.9642662559243566

In [97]:
adata_ood.obsm["X_scVI"] = vae_128.get_latent_representation(adata_ood)
adata_ood.obsm["reconstruction"] = vae_128.get_reconstructed_expression(adata_ood, give_mean=True)
r2_ood = r2_score(np.mean(np.log1p(adata_ood.obsm["reconstruction"]), axis=0), np.array(np.mean(np.log1p(adata_ood.layers["counts"]), axis=0)).squeeze())


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [98]:
r2_ood

0.9641045791474678

In [99]:
adata_test.obsm["X_scVI"] = vae_128.get_latent_representation(adata_test, give_mean=True)
adata_test.obsm["reconstruction"] = vae_128.get_reconstructed_expression(adata_test, give_mean=True)
r2_score(np.mean(np.log1p(adata_test.obsm["reconstruction"]), axis=0), np.array(np.mean(np.log1p(adata_test.layers["counts"]), axis=0))[0,:])

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


0.9636889942226385

In [100]:
adata_train.write(os.path.join(output_dir, "adata_train_300_with_scvi_128.h5ad"))
adata_test.write(os.path.join(output_dir, "adata_test_300_with_scvi_128.h5ad"))
adata_ood.write(os.path.join(output_dir, "adata_ood_300_with_scvi_128.h5ad")) 

In [101]:
with open(os.path.join(save_path, "combosciplex_vae_128.pkl"), 'wb') as f:
    cloudpickle.dump(vae_128, f)

In [103]:
adata_ood.obsm["reconstruction"]

array([[2.70341277e-01, 1.04364716e-02, 6.27671257e-02, ...,
        1.86214180e-04, 1.99079135e-04, 7.90468475e-04],
       [3.03041767e-02, 7.46987155e-03, 3.20437476e-02, ...,
        1.23253209e-04, 1.15279734e-04, 4.83243843e-04],
       [2.35914066e-01, 3.53282019e-02, 3.83374281e-02, ...,
        2.63193186e-04, 3.03605659e-04, 1.31777814e-03],
       ...,
       [2.24414438e-01, 1.74645148e-02, 2.58226786e-02, ...,
        1.80731833e-04, 1.93073807e-04, 7.93089392e-04],
       [3.05246174e-01, 2.60791164e-02, 5.30843772e-02, ...,
        4.08593682e-04, 3.48757545e-04, 7.48134858e-04],
       [1.31517172e-01, 3.75035731e-03, 1.65203735e-02, ...,
        7.83306459e-05, 9.19956728e-05, 2.81815912e-04]], dtype=float32)

In [90]:
juniper.latent.model.FactorVI.setup_anndata(adata_train)
vae_64 = juniper.latent.model.FactorVI(adata_train, n_latent = 128, encoder_layers= [1024, 1024, 1024], decoder_layers = [1024, 1024, 1024], disentangling_weight=0.1, kld_weight=1.0)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot_scvi/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [91]:
vae_64.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=20,
)

array([  6.,   1.,   1.,   1.,   1.,   1.,   2.,   1.,   1.,   2.,   1.,
         1.,   1.,   1.,   1.,   3.,   1.,   2.,   3.,   1.,   1.,   1.,
         1.,   1.,   1.,   2.,   2.,   1.,   4.,   1.,   1.,   1.,   1.,
         1.,   1.,   2.,   1.,   2.,   1.,   2.,   1.,   1.,   2.,   1.,
         1.,   1.,   1.,   1.,   2.,   1.,   1.,   1.,   2.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
         2.,   1.,   1.,  21.,   1.,   1.,   1.,   4.,   1.,   1.,   3.,
         1.,   1., 183.,   1.,   1.,   1.,  10.,   1.], dtype=float32)

In [92]:
adata_test.obsm["X_scVI"].shape

(3100, 256)

In [108]:
np.unique(adata_train.X[0].A, return_counts=True)

(array([0.       , 1.0295198, 1.5259349, 1.8561672, 2.103998 , 2.4679577,
        2.944292 , 3.6582692, 5.8001494], dtype=float32),
 array([1915,   64,   12,    3,    2,    1,    1,    1,    1]))

In [109]:
adata_train.obsm["X_scVI"][0]

array([-0.49189755, -1.476129  , -0.39328223,  0.48536682,  0.15560514,
       -0.59958494,  0.82758117,  0.03984398,  2.0268652 ,  0.22389063,
       -0.24533376,  0.6328336 ,  0.03190783,  0.11749787, -0.8508341 ,
       -1.3484602 , -0.66296947, -1.5698707 , -0.9352404 , -0.5169273 ,
        1.3213114 ,  0.15231681,  0.6270661 , -1.853128  ,  0.37435183,
       -1.1998657 ,  2.5232925 ,  0.99640566, -1.0628371 , -0.942091  ,
       -0.703772  , -0.28993142], dtype=float32)